In [23]:
#LLM

In [1]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pypdf import PdfReader

reader = PdfReader("combined.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]
 
# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(pdf_texts[0])

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)


Common health tests by gender and ageGenderAge RangeTestsMen18-39BLOOD PRESSURE SCREENINGCHOLESTEROL SCREENINGDIABETES SCREENINGSKIN SELF-EXAM
Men40-64BLOOD PRESSURE SCREENINGCHOLESTEROL SCREENINGCOLORECTAL CANCER SCREENINGDIABETES SCREENINGLUNG CANCER SCREENINGOSTEOPOROSIS SCREENINGPROSTATE CANCER SCREENINGSKIN EXAMTESTICULAR EXAM
Men65 and olderABDOMINAL AORTIC ANEURYSM SCREENINGBLOOD PRESSURE SCREENINGCHOLESTEROL SCREENINGCOLORECTAL CANCER SCREENINGDIABETES SCREENINGHEARING TESTLUNG CANCER SCREENINGOSTEOPOROSIS SCREENINGPROSTATE CANCER SCREENINGFor men 70 or older, most recommendations are against screening.Women18-39BLOOD PRESSURE SCREENINGBREAST CANCER SCREENINGCERVICAL CANCER SCREENINGCHOLESTEROL SCREENINGDIABETES SCREENING
Women40-64Blood PressureBreast CancerCervical CancerCholestrolCOLORECTAL CANCER SCREENINGDIABETES SCREENINGLUNG CANCER SCREENINGOSTEOPOROSIS SCREENINGSKIN EXAM


In [10]:
pdf_texts

['Common health tests by gender and ageGenderAge RangeTestsMen18-39BLOOD PRESSURE SCREENINGCHOLESTEROL SCREENINGDIABETES SCREENINGSKIN SELF-EXAM\nMen40-64BLOOD PRESSURE SCREENINGCHOLESTEROL SCREENINGCOLORECTAL CANCER SCREENINGDIABETES SCREENINGLUNG CANCER SCREENINGOSTEOPOROSIS SCREENINGPROSTATE CANCER SCREENINGSKIN EXAMTESTICULAR EXAM\nMen65 and olderABDOMINAL AORTIC ANEURYSM SCREENINGBLOOD PRESSURE SCREENINGCHOLESTEROL SCREENINGCOLORECTAL CANCER SCREENINGDIABETES SCREENINGHEARING TESTLUNG CANCER SCREENINGOSTEOPOROSIS SCREENINGPROSTATE CANCER SCREENINGFor men 70 or older, most recommendations are against screening.Women18-39BLOOD PRESSURE SCREENINGBREAST CANCER SCREENINGCERVICAL CANCER SCREENINGCHOLESTEROL SCREENINGDIABETES SCREENING\nWomen40-64Blood PressureBreast CancerCervical CancerCholestrolCOLORECTAL CANCER SCREENINGDIABETES SCREENINGLUNG CANCER SCREENINGOSTEOPOROSIS SCREENINGSKIN EXAM',
 'Women65 and olderBlood PressureBreast CancerCervical CancerCholestrolCOLORECTAL CANCER SCRE

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [12]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=400,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(f"\nTotal chunks: {len(character_split_texts)}")


Total chunks: 15


In [15]:
type(character_split_texts[0])

str

In [9]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(f"\nTotal chunks: {len(token_split_texts)}")

c:\Users\admin\Desktop\navikenz\rag\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\admin\Desktop\navikenz\rag\myenv\lib\site-packages\huggingface_hub\file_download.py:1135: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Total chunks: 15


In [7]:
print(token_split_texts[10])

levelglucosespecialized thyroid tests like thyroid receptor antibody ( trab ) or thyroid stimulating immunoglobulin ( tsi ) radiology : ultrasoundthyroid scanradioactive iodine uptake test


In [8]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[[0.025213811546564102, -0.036502037197351456, -0.007709131576120853, -0.02681208960711956, 0.0034980291966348886, -0.039768870919942856, 0.08375898748636246, 0.025101959705352783, -0.05233593285083771, -0.01619298942387104, -0.008034572936594486, -0.07158831506967545, 0.010107059963047504, 0.09474480897188187, -0.041657671332359314, -0.09227564930915833, 0.00759788416326046, -0.03248719125986099, 0.022646356374025345, -0.07976178079843521, -0.09007631987333298, 0.015864897519350052, 0.015255708247423172, -0.006999630481004715, 0.00773459579795599, -0.06359231472015381, -0.006879271473735571, -0.06388843804597855, -0.01772714965045452, 0.04101147502660751, -0.02807280607521534, 0.011831266805529594, -0.021981487050652504, -0.0012770319590345025, 0.10894828289747238, 0.016105692833662033, 0.014710295014083385, 0.01396133005619049, -0.025484316051006317, -0.0033905054442584515, 0.008566752076148987, -0.17928573489189148, -0.026755210012197495, -0.037926264107227325, -0.048025112599134445

In [9]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("combined", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

15

In [10]:
query = 'A 69 year old female with difficulty in breathing'

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(document)
    print('\n')

heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat


( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath during exercise or activityeasily tiring during exercise or activityswelling of the hands, ankles cholestrol tests : total cholestrol, ldl, hdl, tryglycerides, non hdl cholestrol. hs - crplp ( a ) plasma ceramidesbnptroponin telectrocardiogramechocardiogramtmtosteoporosisback pain, caused by a broken or collapsed


hyperthyroidismanxietydifficulty concentratingfatiguefrequent bowel movementsgoiter ( visibly enlarged thyroid gland ) or thyroid noduleshair losshand tremorheat intoleranceincreased ap

In [11]:
import os

# from decouple import config
from openai import AzureOpenAI
from openai import OpenAI
from dotenv import load_dotenv
client = AzureOpenAI(
    api_key='32d0ee2f0fa74a60be53d08408584273',
    api_version="2023-10-01-preview",
    azure_endpoint='https://digitalsolutionnrish.openai.azure.com/',
)
# response = client.chat.completions.create(
#     model="gpt-4-32k",  # model = "deployment_name".
#     messages=[
#         {"role": "system", "content": "You are a python instructor."},
#         {"role": "user", "content": "print a statement like hello world in python"},
#     ],
# )

# print(response.choices[0].message.content)

In [12]:
#Agent


In [13]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [14]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [15]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [16]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [17]:
patient_prompt = """You are a helpful medical assistant. For a patient having patient description as {patient_descriptiom} and provided list of medical tests {medical_tests}. Your task is to generate a single useful question asked to the patient so that you can do diagnosis and suggest the patient some medical tests.You will be provided more info later

"""

In [18]:
deciding_prompt = """
You are a helpful medical assistant. For a patient on being asked a question as {question}. The patient answered as {answer}.Output what conclusions you draw from it based on the information given as {info}.Stick strictly to the information given
"""

In [19]:
end_prompt = """
You are a helpful medical assistant . You will be given a patient description as {patient_description} . Your task is to check whether the patient description is enough to suggest one particular test from the given list as {list} or do you need further information if yes output integer 0 if you need more information or Output the test by comma seperated values 
"""

In [20]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")
class AgentState(TypedDict):
    
    patient_des:List[str]
    questions:List[str]
    answers:List[str]
    content:str
    boolean:int
    current_state:str
    
model="gpt-4-32k"

In [21]:
def patient_des_node(state:AgentState):
    print('Patient_node')
    print('------------------------')
    pd = ''.join(state['patient_des'])
    
    content = state['content']
    messages = [
        {
            'role':'system',
            'content':patient_prompt
            
             
        },
        {
            'role':'user',
            'content':f'patient_description:{pd} .\n medical_tests:{content}'
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
        
    )
    content = response.choices[0].message.content
    x = state['questions']  or []
    
    x.append(content)
    return {'questions':x}
    
    

In [22]:
def question_ans(state:AgentState):
    print('Question')
    print('------------------------')
    que = state['questions'][-1]
    ans = state['answers'][-1]
   
    cont = state['content']
    messages = [
        {
            'role':'system',
            'content':deciding_prompt
            
        },
        {
            'role':'user',
            'content':f'question:{que} .\n answer:{ans} .\n info: {cont}'
        }
    
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
        
    )
    content = response.choices[0].message.content
    x = state['answers'] or []
    x.append(content)
    print(x)
    y = state['patient_des'][0]
    y += content
    print(y)
    
    return {'answers':x,'patient_des':[y]}


In [23]:
def end_node(state:AgentState):
    print('End')
    print('------------------------')
    print(state['patient_des'])
    p = state['patient_des']
    l = state['content']
    messages = [
        {
            'role':'system',
            'content': end_prompt
        },
        {
            'role':'user',
            'content':f'patient_description:{p} .\n list:{l}'
        }
        
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
        
    )
    content = response.choices[0].message.content
    if content=='0':
       
       return {'boolean':[0,content]}
    else:
        answers = state['answers']
        
        return {'boolean':[1,content] }
    
    

In [24]:
builder = StateGraph(AgentState)

In [25]:
builder.add_node('patient_description',patient_des_node)
builder.add_node('ques_ans',question_ans)
builder.add_node('end_node',end_node)


In [26]:
builder.set_entry_point('patient_description')

In [27]:
builder.add_conditional_edges(
    'end_node',
    lambda state: state['boolean'][0],  # Use the value from 'next_node'
    
    {1: END, 0: 'patient_description'}
    
    
)

In [28]:
builder.add_edge('patient_description','ques_ans')
builder.add_edge('ques_ans','end_node')


In [29]:
graph = builder.compile(checkpointer=memory,interrupt_before=['ques_ans'])

In [212]:
thread = {"configurable": {"thread_id": "16"}}
s  =  graph.stream({
    'patient_des': ['A 69 year old female with difficulty in breathing'],
    'content': retrieved_documents,
    'questions': ['None'],
    'answers': ['None'],
    'boolean': 0
}, thread)




In [213]:
list(graph.get_state(thread).values)
    

[]

In [214]:
thread

{'configurable': {'thread_id': '16'}}

In [215]:
for i in s:
    print(i.keys())
    if str(i.keys()) == "dict_keys(['patient_description'])":
    
        print(i['patient_description']['questions'][-1],len(i['patient_description']['questions']))

Patient_node
------------------------
dict_keys(['patient_description'])
Have you experienced any recent weight changes, chest discomfort, or increased sensitivity to cold along with your difficulty in breathing? 2


In [216]:
graph.get_state(thread).values

{'patient_des': ['A 69 year old female with difficulty in breathing'],
 'questions': ['None',
  'Have you experienced any recent weight changes, chest discomfort, or increased sensitivity to cold along with your difficulty in breathing?'],
 'answers': ['None'],
 'content': ['heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat',
  '( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath during exercise or activityeasily tiring during exercise or activityswelling of the hands, ankles cholestrol tests : total cholestrol, ldl, hdl, tryglycerides, non hdl cholestrol. hs - crplp ( a ) plasma ceramidesbnptroponi

In [176]:
current_values = graph.get_state(thread)

In [177]:
current_values

StateSnapshot(values={'patient_des': ['A 69 year old female with difficulty in breathing'], 'questions': ['None', 'Considering your age and symptoms of difficulty in breathing, have you noticed any other symptoms lately like experiencing pain, numbness or coldness in your arms or legs, racing or slow heartbeat, getting easily tired during exercise, or showing signs like hair loss, hand tremor, weight changes, menstrual irregularity or any bone pain?'], 'answers': ['None'], 'content': ['heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat', '( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath during exe

In [178]:
current_values.values['answers'].append('no')

In [179]:
graph.get_state(thread).next

('ques_ans',)

In [180]:
graph.update_state(thread,current_values.values)


{'configurable': {'thread_id': '10',
  'thread_ts': '1ef44f00-4ea7-6a1a-8002-19275b378531'}}

In [181]:
thread

{'configurable': {'thread_id': '10'}}

In [182]:
graph.get_state(thread)

StateSnapshot(values={'patient_des': ['A 69 year old female with difficulty in breathing'], 'questions': ['None', 'Considering your age and symptoms of difficulty in breathing, have you noticed any other symptoms lately like experiencing pain, numbness or coldness in your arms or legs, racing or slow heartbeat, getting easily tired during exercise, or showing signs like hair loss, hand tremor, weight changes, menstrual irregularity or any bone pain?'], 'answers': ['None', 'no'], 'content': ['heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat', '( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath duri

In [183]:
for event in graph.stream(None, thread):
    for v in event.values():
        print(v)

Question
------------------------
['None', 'no', "Based on the patient's negative response to the symptoms related to heart diseases, hyperthyroidism, hypothyroidism, and bone-related issues such as osteoporosis, there seems to be no immediate concern regarding these conditions. However, it should be noted that absence of these symptoms does not entirely rule out these conditions since many conditions can be asymptomatic in their early stages. Regular check-ups are required to ensure optimal health."]
A 69 year old female with difficulty in breathingBased on the patient's negative response to the symptoms related to heart diseases, hyperthyroidism, hypothyroidism, and bone-related issues such as osteoporosis, there seems to be no immediate concern regarding these conditions. However, it should be noted that absence of these symptoms does not entirely rule out these conditions since many conditions can be asymptomatic in their early stages. Regular check-ups are required to ensure optim

In [96]:
#recursive code



In [107]:
thread = {"configurable": {"thread_id": "1"}}
s  =  graph.stream({
    'patient_des': ['A 69 year old female with difficulty in breathing'],
    'content': retrieved_documents,
    'questions': ['None'],
    'answers': ['None'],
    'boolean': 0
}, thread)


In [108]:
graph.get_state(thread).values['answers']

['None',
 'no',
 "The patient does not have a history of heart diseases or thyroid conditions and has not recently had a cholesterol check or a bone density test. This suggests that they have not experienced symptoms related to these conditions such as chest pain, irregular heartbeat, anxiety, muscle weakness, or unexpected changes in weight or body structure. They also have not undergone testing for issues related to cholesterol and bone density. But because they haven't had recent tests, their current cholesterol levels and bone health are unknown. Thus no health conditions can be identified at this point. The patient's health status should continue to be monitored and regular tests are advised in the future."]

In [217]:

def run(graph,thread,patient_description,list_tests):
    if len(list(graph.get_state(thread).values))==0:
        s = graph.stream({
        'patient_des': [patient_description],
        'content': list_tests,
        'questions': ['None'],
        'answers': ['None'],
        'boolean': 0
        }, thread)
        return s
    else:
        s = graph.stream(None,thread)
        return s
                
                
        



In [235]:
thread = {"configurable": {"thread_id": "20"}}
s  =  graph.stream({
    'patient_des': ['A 69 year old female with difficulty in breathing'],
    'content': retrieved_documents,
    'questions': ['None'],
    'answers': ['None'],
    'boolean': 0
}, thread)

In [233]:
input = __builtins__.input
def recursive_function_1(graph,thread,patient_description,list_tests):
    x = run(graph,thread,patient_description,list_tests)
    for i in x:
        print(i.keys())
        if str(i.keys()) == "dict_keys(['patient_description'])":
            ans = input(i['patient_description']['questions'][-1])
        elif(str(i.keys()) == "dict_keys(['end_node'])" and i['end_node']['boolean'][0]==1):
            print('Ans---->',i['end_node']['boolean'][1])
            return i['end_node']['boolean'][1]
            
            
    current_values = graph.get_state(thread)
    current_values.values['answers'].append(ans)
    graph.get_state(thread).next
    graph.update_state(thread,current_values.values)
    recursive_function_1(graph,thread,patient_description,list_tests)
        
        

In [236]:
recursive_function_1(graph,thread,patient_description,list_tests)

Patient_node
------------------------
dict_keys(['patient_description'])
Question
------------------------
['None', 'no', "The patient's negative response to experiencing weight changes, irregular menstrual periods, sensitivity to cold, fatigue or muscle stiffness suggests that they may not have conditions like Hypothyroidism or PCOS which are characterized by these symptoms. Furthermore, they may also not be experiencing Hyperthyroidism as the patient also denies symptoms of frequent bowel movements, increased appetite, or heat intolerance. Additionally, the lack of reported fatigue or muscle stiffness also suggests they may not have conditions like heart diseases or Osteoporosis. This is strictly based on the information provided and further testing would be required for a complete diagnosis."]
A 69 year old female with difficulty in breathingThe patient's negative response to experiencing weight changes, irregular menstrual periods, sensitivity to cold, fatigue or muscle stiffness s

In [219]:
x = run(graph,thread,patient_description,list_tests)

In [220]:
for i in s:
    print(i.keys())
    if str(i.keys()) == "dict_keys(['patient_description'])":
    
        print(i['patient_description']['questions'][-1],len(i['patient_description']['questions']))

Patient_node
------------------------
dict_keys(['patient_description'])
Have you noticed any changes in your appetite, sleep, or experiencing more sensitivity to cold along with your breathing difficulty? 2


In [221]:
graph.get_state(thread).values

{'patient_des': ['A 69 year old female with difficulty in breathing'],
 'questions': ['None',
  'Have you noticed any changes in your appetite, sleep, or experiencing more sensitivity to cold along with your breathing difficulty?'],
 'answers': ['None'],
 'content': ['heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat',
  '( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath during exercise or activityeasily tiring during exercise or activityswelling of the hands, ankles cholestrol tests : total cholestrol, ldl, hdl, tryglycerides, non hdl cholestrol. hs - crplp ( a ) plasma ceramidesbnptroponin telec

In [222]:
current_values = graph.get_state(thread)
current_values.values['answers'].append('no')
graph.get_state(thread).next

('ques_ans',)

In [224]:
graph.update_state(thread,current_values.values)
graph.get_state(thread).values['answers']

['None', 'no']

In [226]:
y = run(graph,thread,patient_description,list_tests)

In [227]:
for event in y:
    for v in event.values():
        print(v)

Question
------------------------
['None', 'no', "Based on the patient's response, we can conclude that they are not experiencing symptoms associated with conditions such as hypothyroidism (sensitivity to cold, changes in sleep), or hyperthyroidism (changes in appetite, sleep disturbances). The reported breathing difficulty could be associated with heart diseases as it is mentioned in the provided information, but the absence of other symptoms like chest pain, dizziness, and coldness in limbs makes it less conclusive. More tests may be needed to rule out such possibilities."]
A 69 year old female with difficulty in breathingBased on the patient's response, we can conclude that they are not experiencing symptoms associated with conditions such as hypothyroidism (sensitivity to cold, changes in sleep), or hyperthyroidism (changes in appetite, sleep disturbances). The reported breathing difficulty could be associated with heart diseases as it is mentioned in the provided information, but 

Patient_node
------------------------
{'patient_description': {'questions': ['None', 'Considering your difficulty in breathing and your age, can you tell me more about your medical history, including any heart diseases, thyroid issues, or osteoporosis, and when was the last time you had a complete blood panel that includes cholesterol, hormone tests and bone panel?', 'Considering your symptoms and age, have you noticed any visible changes such as weight loss or gain, frequent bowel movements, or any palpitations or restlessness that might suggest thyroid issues or cardiac lessions along with your breathing difficulty?']}}


In [33]:
input = 'no'
s = graph.stream(
    {'answers':[input]},
    thread
)
for i in s:
    print(i)

Patient_node
------------------------
{'patient_description': {'questions': ['None', 'Given your difficulty in breathing, have you also experienced symptoms like chest pain, dizziness, or rapid heartbeat recently? Also, have you noticed changes in your weight, menstrual cycles, or sensitivity to cold?', "Since you're experiencing difficulty in breathing, have you noticed any associated symptoms such as chest pain or discomfort, dizziness, fainting, palpitations or easily getting short of breath during exercise or activity?"]}}


In [ ]:
!pip install pygraphviz

     ---------------------------------------- 0.0/120.8 kB ? eta -:--:--
     ------ ------------------------------ 20.5/120.8 kB 330.3 kB/s eta 0:00:01
     ------------------ ------------------ 61.4/120.8 kB 544.7 kB/s eta 0:00:01
     ------------------------------------ 120.8/120.8 kB 784.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\pygraphviz
      copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-39\pygraphviz
      copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-39\pygraphviz
      copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-39\pygraphviz
      copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-39\pygraphviz
      copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-39\pygraphviz
      creating build\lib.win-amd64-cpython-39\pygraphviz\tests
      copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-39\pygraphviz\tests
      copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64

In [83]:
# def rag(query, retrieved_documents, model="gpt-4-32k"):
#     information = "\n\n".join(retrieved_documents)

#     messages = [
#         {
#             "role": "system",
#             "content": "You are a helpful expert in medical field. Your users are asking questions contained in the medical tests list provided."
#             "You will be given data about patient and related to the information from medical tests list . Your task is to ask neccesary questions to the patient to get to a conclusion and suggest him some tests strictly from the information given.Output the questions in comma seperated values"
#         },
#         {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
#     ]
    
#     response = client.chat.completions.create(
#         model=model,
#         messages=messages,
#     )
#     content = response.choices[0].message.content
#     return content


messages = [
    {
        'role':'system',
        'content':end_prompt
        
    },
    {
        'role':'user',
        'content':"patient_description:'A 69 year old female with difficulty in breathing' .\n list: {retrieved_documents}"
        
    }
    
    
]

In [84]:
response = client.chat.completions.create(
        model="gpt-4-32k",
        messages=messages,
    )
content = response.choices[0].message.content
print(content)

0


In [35]:
def test(state:AgentState):
    
    x = state['answers']
    x.append('1')
    return {'answers':x}

In [36]:
s = AgentState(
    answers=[]
    
)

In [37]:
test(s)

{'answers': ['1']}

In [ ]:
class AgentState(TypedDict):
    patient_des:List[str]
    questions:List[str]
    answers:List[str]
    content:str
    boolean:int
def patient_des_node(state:AgentState):
    pd = ''.join(state['patient_des'])
    content = state['content']
    messages = [
        {
            'role':'system',
            'content':patient_prompt
            
             
        },
        {
            'role':'user',
            'content':f'patient_description:{pd} .\n medical_tests:{content}'
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
        
    )
    content = response.choices[0].message.content
    x = state['questions']  or []
    
    x.append(content)
    return {'questions':x}
def question_ans(state:AgentState):
    que = state['questions'][-1]
    ans = input(que)
    cont = state['content']
    messages = [
        {
            'role':'system',
            'content':deciding_prompt
            
        },
        {
            'role':'user',
            'content':f'question:{que} .\n answer:{ans} .\n info: {cont}'
        }
    
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
        
    )
    content = response.choices[0].message.content
    x = state['answers']
    x.append(content)
    return {'answers':content}
def end_node(state:AgentState):
    p = state['patient_des']
    l = state['content']
    messages = [
        {
            'role':'system',
            'content': end_prompt
        },
        {
            'role':'user',
            'content':f'patient_description:{p} .\n list:{l}'
        }
        
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
        
    )
    content = response.choices[0].message.content
    if content==0:
       print(state['boolean'])
       return {'boolean':0}
    else:
        answers = state['answers']
        
        return {'boolean':1 }
    
def check(state:AgentState):
    if state['boolean']==0:
        return 0
    else:
        return 1    
    
builder = StateGraph(AgentState)
builder.add_node('patient_description',patient_des_node)
builder.add_node('ques_ans',question_ans)
builder.add_node('end_node',end_node)
builder.add_node('check',check)
builder.set_entry_point('patient_description')
builder.add_conditional_edges(
    'check',
    check,
    {1:END,0:'ques_ans'}
    
    
)
builder.add_edge('patient_description','ques_ans')
builder.add_edge('ques_ans','end_node')
builder.add_edge('end_node','check')
graph = builder.compile(checkpointer=memory)
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'patient_des':['A 26 year old female with short term memory loss'],
    'content':retrieved_documents,
    'questions':[],
    'answers':[],
    
    'boolean':0
    
}, thread):
    print(s)
# patient_des:List[str]
#     questions:List[str]
#     answers:List[str]
#     content:str
#     boolean:int

In [74]:
from typing import List, TypedDict

class AgentState(TypedDict):
    patient_des: List[str]
    questions: List[str]
    answers: List[str]
    content: str
    boolean: int

def patient_des_node(state: AgentState):
    pd = ' '.join(state['patient_des'])  # Correct concatenation
    content = state['content']
    messages = [
        {
            'role': 'system',
            'content': patient_prompt
        },
        {
            'role': 'user',
            'content': f'patient_description: {pd}.\nmedical_tests: {content}'
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    content = response.choices[0].message.content  # Access content with dot notation
    questions = state['questions'] or []
    questions.append(content)
    return {'questions': questions}

def question_ans(state: AgentState):
    que = state['questions'][-1]
    # Simulate an answer input, replace with actual input method
    ans = input(que)
    cont = state['content']
    messages = [
        {
            'role': 'system',
            'content': deciding_prompt
        },
        {
            'role': 'user',
            'content': f'question: {que}.\nanswer: {ans}.\ninfo: {cont}'
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    content = response.choices[0].message.content  # Access content with dot notation
    answers = state['answers']
    answers.append(content)
    return {'answers': answers}

def end_node(state: AgentState):
    p = ' '.join(state['patient_des'])
    l = state['content']
    messages = [
        {
            'role': 'system',
            'content': end_prompt
        },
        {
            'role': 'user',
            'content': f'patient_description: {p}.\nlist: {l}'
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    content = response.choices[0].message.content  # Access content with dot notation
    if content == '0':
        return {'boolean': 0}
    else:
        return {'boolean': 1}

def check(state: AgentState):
    if state['boolean'] == 0:
        return {'next_node': 0}  # Wrapped in dictionary
    else:
        return {'next_node': 1}  # Wrapped in dictionary

# Assuming StateGraph and related methods are defined elsewhere
builder = StateGraph(AgentState)
builder.add_node('patient_description', patient_des_node)
builder.add_node('ques_ans', question_ans)
builder.add_node('end_node', end_node)
builder.add_node('check', check)
builder.set_entry_point('patient_description')
builder.add_conditional_edges(
    'check',
    lambda state: state['next_node'],  # Use the value from 'next_node'
    {1: END, 0: 'ques_ans'}
)
builder.add_edge('patient_description', 'ques_ans')
builder.add_edge('ques_ans', 'end_node')
builder.add_edge('end_node', 'check')
graph = builder.compile(checkpointer=memory)
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'patient_des': ['A 26 year old female with short term memory loss'],
    'content': retrieved_documents,
    'questions': [],
    'answers': [],
    'boolean': 0
}, thread):
    print(s)


{'patient_description': {'questions': ['"Could you describe the extent of your short term memory loss? Do you find it impacting your daily activities and is there a pattern to it?"']}}
{'ques_ans': {'answers': ['The patient experiences short term memory loss, impacting his daily activities. However, the information given does not immediately suggest any medical tests or conditions directly associated with this symptom. Further observation, information or consulting with a healthcare professional would be advisable.']}}
{'end_node': {'boolean': 0}}


InvalidUpdateError: Must write to at least one of ['patient_des', 'questions', 'answers', 'content', 'boolean']

In [43]:
thread = {"configurable": {"thread_id": "542"}}
s  =  graph.stream({
    'patient_des': ['A 69 year old female with difficulty in breathing'],
    'content': retrieved_documents,
    'questions': ['None'],
    'answers': ['None'],
    'boolean': 0
}, thread)

In [44]:
for i in s:
    for j in i.values():
        print(j)

Patient_node
------------------------
{'questions': ['None', 'Considering your symptoms, have you ever experienced severe chest pain, dizziness, fainting, or irregular heartbeats? Additionally, can you share any prior history and frequency of any thyroid disorders, osteoporosis, or cardiovascular diseases in your medical history?']}


In [32]:
graph.get_state(thread).values

{'patient_des': ['A 69 year old female with difficulty in breathing'],
 'questions': ['None',
  "With your complaints of difficulty in breathing, can you please describe if you're also experiencing other symptoms such as chest pain, dizziness, irregular heartbeat, constipation, changes in weight or menstrual cycle, or any abnormal changes in your skin and hair?"],
 'answers': ['None'],
 'content': ['heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat',
  '( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath during exercise or activityeasily tiring during exercise or activityswelling of the hands, ankle

In [45]:
print(str(graph.get_state(thread).next) )

('ques_ans',)


In [33]:
current_values = graph.get_state(thread)

In [34]:
current_values

StateSnapshot(values={'patient_des': ['A 69 year old female with difficulty in breathing'], 'questions': ['None', "With your complaints of difficulty in breathing, can you please describe if you're also experiencing other symptoms such as chest pain, dizziness, irregular heartbeat, constipation, changes in weight or menstrual cycle, or any abnormal changes in your skin and hair?"], 'answers': ['None'], 'content': ['heart diseaseschest pain, chest tightness, chest pressure and chest discomfort ( angina ) shortness of breathpain in the neck, jaw, throat, upper belly area or backpain, numbness, weakness or coldness in the legs or arms if the blood vessels in those body areas are narrowedchest pain or discomfortdizzinessfainting ( syncope ) or near faintingfluttering in the chestlightheadednessracing heartbeat', '( tachycardia ) shortness of breathslow heartbeat ( bradycardia ) easily getting short of breath during exercise or activityeasily tiring during exercise or activityswelling of th

In [35]:
current_values.values['answers'].append('no')

In [36]:
graph.get_state(thread).next

('ques_ans',)

In [42]:
print(str(graph.get_state(thread).next) )

('ques_ans',)
